# 필요 패키지

In [53]:
# 필요 패키지
import pandas as pd
import numpy as np
import re

# 데이터 불러오기

In [ ]:
# 데이터 불러오기
df = pd.read_csv('데이터셋 경로 작성', sep=',', encoding='utf-8', index_col=0)
df.head()

# 해시태그 추출

In [55]:
# 해시태그 추출
hashtag_regex = "#([0-9a-zA-Z가-힣]*)"
p = re.compile(hashtag_regex)
df["hash_tag"] = df["content_text"].apply(lambda x : p.findall(str(x)))
# 해시태그 수 세기
df['hash_tag_num'] = df['hash_tag'].apply(lambda x : len(x))

# mention 추출

In [56]:
# mention 추출
mention_regex = "@([0-9a-zA-Z가-힣_.]*)"
p = re.compile(mention_regex)
df['mention_name'] = df["reply"].apply(lambda x : p.findall(str(x)))
# mention 수 세기
df['mention'] = df['mention_name'].apply(lambda x : len(x))

# 광고글 판단

## 광고글 판단 함수

In [57]:
# 광고글인지 판단하는 함수
def chk_adv(main_txt):
    temp = 0
    promo_word = ['광고', '협찬', '제품제공', 'AD', 'GIFTED', '제공', '제품협찬', '유료광고', '상업광고', '금전적']
    for w in promo_word:
        if w in main_txt:
            temp = 1
        if temp ==1:
            break
    return temp

# 전처리 함수

In [58]:
# 전처리 함수(글자, 영어, 숫자만 남기기)
def text_preprocessor(s):
    import re
    
    ## (1) [], (), {}, <> 괄호 제거
    s = str(s).replace('(', ' ')
    s = str(s).replace(')', ' ')
    s = str(s).replace('[', ' ')
    s = str(s).replace(']', ' ')
    s = str(s).replace('<', ' ')
    s = str(s).replace('>', ' ')
    s = str(s).replace('{', ' ')
    s = str(s).replace('}', ' ')


    hashtag_regex = "#([0-9a-zA-Z가-힣_.]*)"
    s = re.sub(pattern=hashtag_regex, repl='', string=s)
    
    call_regex = "@([0-9a-zA-Z가-힣_.]*)"
    s = re.sub(pattern=call_regex, repl='', string=s)


    ## (2) '...외', '...총' 제거하기
    s = s.replace('...외', ' ')
    s = s.replace('...총', ' ')
    
    ## (3) 특수문자 제거
    pattern = r'[^a-zA-Z가-힣]'
    s = re.sub(pattern=pattern, repl=' ', string=s)
    
    ## (4) 단위 제거: cm, km, etc.
    units = ['mm', 'cm', 'km', 'ml', 'kg', 'g']
    for unit in units:
        s = s.lower() # 대문자를 소문자로 변환
        s = s.replace(unit, '')
        
    # (5) 공백 지워서 한줄로 합치기
    s = s.replace(' ', '')
    
    


    return s

## 게시글 전처리

### 게시글 전처리함수 적용 (필요시 사용)

In [59]:
# 게시글 전처리함수 적용
df['pre_content'] = df['content_text'].apply(lambda s: text_preprocessor(s))

### 게시글 대문자화 (브랜드명 추출할 때 사용하기 위해)

In [60]:
df['pre_content'] = df.loc[:, 'pre_content'].apply(lambda x: x.upper())

### 해시태그 대문자화

In [61]:
df['hash_tag'] = df.loc[:,'hash_tag'].apply(lambda x : str(x).upper())

## 광고글 체크 함수 적용

In [62]:
df["promotion"] = df["pre_content"].apply(lambda x : chk_adv(str(x)))

## 광고글만 따로 df

In [63]:
# 광고글만 따로 df 만들기
df_pro = df[df['promotion']==1]

## 일상글만 따로 df

In [ ]:
# 일상글만 따로 df 만들기
df_norm = df[df['promotion']==0]

## 댓글 전처리

### 댓글 전처리함수 적용

In [64]:
# 댓글 전처리함수 적용
df_pro['pre_reply'] = df_pro['reply'].apply(lambda s: text_preprocessor(s))

/var/folders/kl/cdc24_t10v95bx7k95vy_n_40000gn/T/ipykernel_50932/3307771747.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pro['pre_reply'] = df_pro['reply'].apply(lambda s: text_preprocessor(s))


### 댓글 대문자화

In [65]:
# 전처리된 'reply' 대문자화
df_pro['pre_reply'] = df_pro.loc[:, 'pre_reply'].apply(lambda x: x.upper())

/var/folders/kl/cdc24_t10v95bx7k95vy_n_40000gn/T/ipykernel_50932/3428017239.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pro['pre_reply'] = df_pro.loc[:, 'pre_reply'].apply(lambda x: x.upper())


## 한글 포함 여부 체크 함수

In [66]:
# 한글 포함 여부 체크 함수
def isKorean(text):
    hangul = re.compile('[\u3131-\u3163\uac00-\ud7a3]+')  
    result = hangul.findall(text)
    if len(result) > 0:
        # 한글이 있다면 True 반환
        return True
    else:
        # 한글이 없다면 False 반환
        return False

### 전처리된 댓글에 한글 포함 여부 체크

In [67]:
# 전처리된 'reply'에 한글 포함 여부 체크 함수 적용
df_pro['hangul_exist'] = df_pro['pre_reply'].apply(lambda s: isKorean(s))

/var/folders/kl/cdc24_t10v95bx7k95vy_n_40000gn/T/ipykernel_50932/118098914.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pro['hangul_exist'] = df_pro['pre_reply'].apply(lambda s: isKorean(s))


# 댓글에 한글이 없거나 글이 없는 경우 drop

In [68]:
df_pro = df_pro[df_pro['hangul_exist'] == True]

In [ ]:
df_pro